In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from keras.models import load_model
model = load_model('classifier_model.h5')

In [11]:
def quantile_25(x):
    return x.quantile(0.25)
def quantile_5(x):
    return x.quantile(0.5)
def quantile_75(x):
    return x.quantile(0.75)
def quantile_8(x):
    return x.quantile(0.8)
def quantile_9(x):
    return x.quantile(0.9)
def triptime(x):
    return x.max()

def plot_roc_curve(fpr, tpr):  
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [12]:
def generate_feature(df):
    df.sort_values(['bookingID','second'],inplace=True)
    radToDeg = 57.2957786
    df['roll'] = np.arctan2(df['acceleration_y'],df['acceleration_z']) * radToDeg
    df['pitch'] = np.arctan(-df['acceleration_x']/np.sqrt((df['acceleration_y']**2)+(df['acceleration_z']**2))) * radToDeg
    df['compAngleX'] = 0.93 * (df['roll'] + df['gyro_x']/131 ) + 0.07 * df['roll']
    df['compAngleY'] = 0.93 * (df['pitch'] + df['gyro_x']/131 ) + 0.07 * df['pitch']
    df['Bearing_diff']=df['Bearing'].diff()
    df.loc[df['second']==0,['Bearing_diff']]= 0
    df['Speed_diff']=df['Speed'].diff()
    df.loc[df['second']==0,['Speed_diff']]= 0
    df['distance'] = df['Speed'] * df['second'].diff()
    df['distance'][df['second'] == 0] = 0
    model_df_1 = df.pivot_table(index='bookingID',values = ['distance'], aggfunc=sum)
    model_df_2 = df.pivot_table(index='bookingID',values = ['second'], aggfunc=max)
    model_df_3 = df.pivot_table(index='bookingID',values=['Speed','Bearing','compAngleX','compAngleY','Bearing_diff','Speed_diff'],aggfunc=[quantile_25,quantile_5,quantile_75,quantile_8,quantile_9])
    model_df_4 = df.pivot_table(index='bookingID',values=['Speed','Bearing','compAngleX','compAngleY','Bearing_diff','Speed_diff'],aggfunc=[np.mean,np.std,np.min,np.max])
    model_df = model_df_1.join([model_df_2,model_df_3,model_df_4])
    return model_df


In [13]:
test = pd.read_csv('test_trip.cvs')

In [14]:
test_feature = generate_feature(test)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [15]:
test_feature 

distance  second  (quantile_25, Bearing)  \
bookingID                                               
1.0        6844.87121  1034.0                    50.0   

           (quantile_25, Bearing_diff)  (quantile_25, Speed)  \
bookingID                                                      
1.0                               -1.0                   0.0   

           (quantile_25, Speed_diff)  (quantile_25, compAngleX)  \
bookingID                                                         
1.0                        -0.227863                 100.847996   

           (quantile_25, compAngleY)  (quantile_5, Bearing)  \
bookingID                                                     
1.0                         1.365338                  110.0   

           (quantile_5, Bearing_diff)  ...  (amin, Speed)  (amin, Speed_diff)  \
bookingID                              ...                                      
1.0                               0.0  ...            0.0           -4.099883   

           (amin, compAngleX)  (amin, compAngleY)  (amax, Bearing)  \
bookingID                                                            
1.0                 82.203521          -24.178214            337.0   

           (amax, Bearing_diff)  (amax, Speed)  (amax, Speed_diff)  \
bookingID                                                            
1.0                        39.0      21.882141            4.188555   

           (amax, compAngleX)  (amax, compAngleY)  
bookingID                                          
1.0                122.460492           29.361171  

[1 rows x 56 columns]

In [20]:
predict = model.predict(test_feature)

In [24]:
print('Non dangerous driving') if (predict>0.5) else print('Dangerous driving')

Dangerous driving
